# HyperDrive Run Recovery
In this notebook, we recover into Python the HyperDrive run that was created in the previous notebook, and use it to find the best child run discovered by the HyperDrive search.

The steps in this notebook are
- [import libraries](#import),
- [read in the Azure ML workspace](#workspace),
- [recover a run](#recover), and
- [get the results](#results).

## Imports  <a id='import'></a>

In [ ]:
import os
import shutil
import json
import time
import pandas as pd
from azureml.core import Workspace, Experiment, Run, get_run
from azureml.widgets import RunDetails
import azureml.core
from msrest.exceptions import HttpOperationError
from get_auth import get_auth
print('azureml.core.VERSION={}'.format(azureml.core.VERSION))

## Read in the Azure ML workspace  <a id='workspace'></a>
Read in the the workspace created in a previous notebook.

In [ ]:
auth = get_auth()
ws = Workspace.from_config(auth=auth)
ws_details = ws.get_details()
print('Name:\t\t{}\nLocation:\t{}'
      .format(ws_details['name'],
              ws_details['location']))

## Recover the run  <a id='recover'></a>
Get an experiment that ran the search.

In [ ]:
exp = Experiment(workspace=ws, name='hypetuning')

Get the ID of the HyperDrive run created in the last notebook. That ID was printed with the run when it was submitted in the previous notebook, and we also saved it in a file. You can also find that ID in Azure Portal on your experiment's page. To see it, you may need to add a `RunId` column to the experiment's table of runs.

In [ ]:
run_id_path = "run_id.txt"
with open(run_id_path, "r") as fp:
    run_id = fp.read()
run_id

Use the ID of the HyperDrive run to get a handle to it.

In [ ]:
run = get_run(exp, run_id, rehydrate=True)
run

## Get the results <a id='results'></a>
Get the metrics logged with each run.

In [ ]:
run_metrics = run.get_metrics(recursive=True)

Get a series with each run's gain.

In [ ]:
run_gain = pd.Series([x["gain@3"][-1] for x in run_metrics.values()], index=run_metrics.keys(), name="gain@3")

Report the maximum gain.

In [ ]:
run_gain.max()

Find the RunId of the best run.

In [ ]:
best_run_id = run_gain.idxmax()
best_run_id

Use it to recover the best run.

In [ ]:
best_run = get_run(exp, best_run_id, rehydrate=True)
best_run

The [next notebook](06_Tear_Down.ipynb) shows how to delete the components of this tutorial.